## Restraunt AI assistant -
 Available tools:
 - get_menu()
 - get_available_tables(date,people)
 - book_table(name, date, time, people)
 - calculate_bill(items)

In [1]:
import os
import gradio
from dotenv import load_dotenv
from openai import OpenAI
from datetime import datetime
import psycopg

In [2]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print("Got the key")

Got the key


In [52]:
current_date =  datetime.now().strftime('%Y-%m-%d')
current_time = datetime.now().strftime('%H:%M:%S')
print(f"Current date - {current_date} and current time is {current_time}")
print(datetime.now())

Current date - 2026-02-27 and current time is 15:58:50
2026-02-27 15:58:50.155665


In [59]:
def get_menu():
    with psycopg.connect("dbname=MyRestaurantDB user=postgres password=Veenamaa@2103") as conn:
            with conn.cursor() as cur:
                cur.execute("SELECT * FROM Menu;")
                rows = cur.fetchall()
    return rows

def get_available_tables(date, time, people):
    with psycopg.connect("dbname=MyRestaurantDB user=postgres password=Veenamaa@2103") as conn:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT t.*
                FROM tables t
                WHERE t."Capacity" >= %s
                AND NOT EXISTS (
                    SELECT 1
                    FROM bookings b
                    WHERE b.table_id = t.id
                    AND b."Date" = %s
                    AND (
                        b."Time" < (%s::time + interval '2 hours')
                        AND
                        (b."Time" + interval '2 hours') > %s::time
                    )
                )
            """,(people,date,time,time))
            rows = cur.fetchall()
    return rows

def book_table(name,date,time,people):
    with psycopg.connect("dbname=MyRestaurantDB user=postgres password=Veenamaa@2103") as conn:
        with conn.cursor() as cur:
            available_tables = get_available_tables(date,time, people)
            if available_tables:
                table_id = available_tables[0][0]
                cur.execute("""
                    INSERT INTO bookings("table_id", "Name", "Date", "Time", "People")
                    VALUES (%s, %s, %s, %s, %s) RETURNING booking_id
                """, (table_id, name, date, time, people))
                booking_id = cur.fetchone()[0]
                conn.commit()
                return f"Table {table_id} booked successfully with booking id - {booking_id} for {name} on {date} at {time}."
            else:
                return "No available tables for the selected date."

def add_to_cart(booking_id, items):
    with psycopg.connect("dbname=MyRestaurantDB user=postgres password=Veenamaa@2103") as conn:
        with conn.cursor() as cur:
            for item, quantity in items.items():
                print(item, quantity)
                cur.execute("INSERT INTO cart(\"booking_id\", \"item\", \"quantity\") VALUES (%s, %s, %s)", (booking_id, item, quantity))
            conn.commit()
            return "Your items are added"

def remove_or_update_cart(booking_id, item, quantity=0):
     with psycopg.connect("dbname=MyRestaurantDB user=postgres password=Veenamaa@2103") as conn:
        with conn.cursor() as cur:
            cur.execute("UPDATE cart SET \"quantity\" = %s WHERE booking_id = %s AND item = %s", (quantity,booking_id, item))
            conn.commit()
            return f"Item quantity for {item} for booking id {booking_id} is udpated to {quantity}"

def calculate_bill(booking_id):
    with psycopg.connect("dbname=MyRestaurantDB user=postgres password=Veenamaa@2103") as conn:
        with conn.cursor() as cur:
            total_price = 0
            cur.execute("SELECT item, quantity from cart where booking_id=%s", (booking_id,))
            rows = cur.fetchall()
            for items in rows:
                print(items)
                cur.execute("SELECT \"Price\" from menu where \"Dish Name\"= %s", (items[0],))
                total_price += (items[1] * (cur.fetchone()[0]))
            return total_price

def cancel_bookings(booking_id):
    with psycopg.connect("dbname=MyRestaurantDB user=postgres password=Veenamaa@2103") as conn:
        with conn.cursor() as cur:
            now = datetime.now()
            cur.execute('SELECT "Date", "Time" from bookings where booking_id=%s',(booking_id,))
            rows = cur.fetchone()
            if rows: 
                date,time = rows
                booking_datetime = datetime.combine(date, time)
                if booking_datetime <= now:
                    return "Too late to cancel"
                else:
                    cur.execute("DELETE FROM bookings where booking_id=%s",(booking_id,))
                    cur.execute("DELETE FROM cart where booking_id=%s", (booking_id,))
                    conn.commit()
                    return f"Booking for booking id {booking_id} canceled successfully"
            else:
                return "No bookings for the booking id found"